In [3]:
!pip install langchain-nvidia-ai-endpoints

In [14]:
# rag.ipynb

# --- IMPORTS ---
import os
from dotenv import load_dotenv
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter
import shutil
import gradio as gr



# --- Initialize NVIDIA LLM ---
nvidia_llm = ChatNVIDIA(model="meta/llama3-8b-instruct", temperature=0.3,nvcf_api_key="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")



# --- SETTINGS ---
DOCS_PATH = "knowledge_base"  # Folder with your Markdown/Text files
CHROMA_DB_PATH = "chroma_db_fitflix_vector_2"

# --- STEP 1: Load Documents ---
loader = DirectoryLoader(DOCS_PATH, glob="**/*.md", loader_cls=TextLoader)
documents = loader.load()
print(f"✅ Loaded {len(documents)} documents.")

# --- STEP 2: Split Text into Chunks ---
text_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=[
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3")
    ]
)
docs = []
for doc in documents:
    docs.extend(text_splitter.split_text(doc.page_content))

print(f"✅ Split into {len(docs)} text chunks.")



✅ Loaded 9 documents.
✅ Split into 49 text chunks.


C:\Users\dandu\anaconda3\envs\llms\Lib\site-packages\langchain_nvidia_ai_endpoints\_common.py:184: UserWarning: An API key is required for the hosted NIM. This will become an error in the future.
  warnings.warn(


In [15]:
#--- STEP 3: Initialize Embeddings and Chroma Vector Store ---
embeddings = NVIDIAEmbeddings(model="nvidia/lama-3_2-nemoretriever-1b-vlm-embed-v1")

# Remove old DB if exists
if os.path.exists(CHROMA_DB_PATH):
    shutil.rmtree(CHROMA_DB_PATH)

vectorstore = Chroma.from_texts([d.page_content for d in docs], embedding=embeddings, persist_directory=CHROMA_DB_PATH)
vectorstore.persist()
print("✅ Chroma Vector Store created.")

# --- STEP 4: Setup RAG Pipeline ---
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

# Minimal response template
prompt_template = """You are Fitflix AI, a concise fitness and nutrition assistant.
Answer the user's question using the context below in **2 sentences maximum**.
If you are unsure, say 'I am not sure about that.'

Context:
{context}

Question: {question}

Answer:
"""

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key="answer")

rag_chain = ConversationalRetrievalChain.from_llm(
    llm=nvidia_llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": PROMPT},
    return_source_documents=True,
    verbose=False
)



C:\Users\dandu\anaconda3\envs\llms\Lib\site-packages\langchain_nvidia_ai_endpoints\_common.py:227: UserWarning: Model nvidia/lama-3_2-nemoretriever-1b-vlm-embed-v1 is unknown, check `available_models`. Inference may fail.
  warnings.warn(


Exception: [404] Not Found
{'_content': b'404 page not found\n', '_content_consumed': True, '_next': None, 'status_code': 404, 'headers': {'Date': 'Mon, 04 Aug 2025 16:45:01 GMT', 'Content-Type': 'text/plain; charset=utf-8', 'Content-Length': '19', 'Connection': 'keep-alive', 'Vary': 'Origin', 'X-Content-Type-Options': 'nosniff'}, 'raw': <urllib3.response.HTTPResponse object at 0x000001F2DA207B20>, 'url': 'https://integrate.api.nvidia.com/v1/embeddings', 'encoding': 'utf-8', 'history': [], 'reason': 'Not Found', 'cookies': <RequestsCookieJar[]>, 'elapsed': datetime.timedelta(microseconds=249842), 'request': <PreparedRequest [POST]>, 'connection': <requests.adapters.HTTPAdapter object at 0x000001F2DA395A10>}

In [6]:
# --- STEP 5: Create Gradio Chat Interface ---
def chat_fn(message, history):
    result = rag_chain.invoke({"question": message})
    answer = result["answer"]
    return answer

with gr.Blocks() as demo:
    gr.Markdown("## 🏋️ Fitflix RAG Chatbot (Powered by NVIDIA)")
    chatbot = gr.Chatbot(height=400)
    msg = gr.Textbox(label="Ask me about fitness, workouts, or diet...")
    clear = gr.Button("Clear Chat")

    def respond(message, chat_history):
        response = chat_fn(message, chat_history)
        chat_history.append((message, response))
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch(share=False)


C:\Users\dandu\AppData\Local\Temp\ipykernel_25204\482280328.py:9: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=400)


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "C:\Users\dandu\anaconda3\envs\llms\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\dandu\anaconda3\envs\llms\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\dandu\anaconda3\envs\llms\Lib\site-packages\gradio\blocks.py", line 2146, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\dandu\anaconda3\envs\llms\Lib\site-packages\gradio\blocks.py", line 1664, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\dandu\anaconda3\envs\llms\Lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return awai